# "2020 February One Day Holding Long Short Portfolio BackTest"
> "2020 February Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [32]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [33]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\20204dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\8K20194dayreturnpred2019train60K.csv')

In [34]:
#hide
df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20200201].loc[df_testp.Date<20200301].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20200201].loc[df_test.Date<20200301].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [35]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [36]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [37]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a
0,2020-02-03,-1.0,2.859980,0.971400,1.014980,1.014360,0.860672
1,2020-02-03,-1.0,2.859980,0.971400,1.014980,1.014360,0.781195
2,2020-02-03,1.0,-0.892751,0.962728,1.014980,1.014360,0.860672
3,2020-02-03,1.0,-0.892751,0.962728,1.014980,1.014360,0.781195
4,2020-02-04,-1.0,4.235264,0.921954,1.026400,1.031375,0.366193
...,...,...,...,...,...,...,...
63,2020-02-25,1.0,-63.613512,4.410129,0.959208,0.949217,0.003874
64,2020-02-26,-1.0,-93.760997,8.545111,0.959208,0.949217,0.007038
65,2020-02-26,-1.0,-93.760997,8.545111,0.959208,0.949217,-0.004138
66,2020-02-26,1.0,-293.470649,-16.532281,0.959208,0.949217,0.007038


In [38]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4

alt.LayerChart(...)

**Long portfolio**

In [39]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [40]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a
0,2020-02-03,1.0,-0.892751,0.991072,1.014980,1.014360,0.907657
1,2020-02-04,1.0,5.778624,1.048343,1.026400,1.031375,0.893016
2,2020-02-05,1.0,-5.646498,0.989148,1.029813,1.034506,1.133167
3,2020-02-06,1.0,-2.618297,0.963249,1.024251,1.024743,0.958787
4,2020-02-07,1.0,3.719168,0.999074,1.031755,1.030881,0.989969
5,2020-02-10,1.0,-7.021549,0.928924,1.033497,1.030864,1.128090
6,2020-02-11,1.0,65.597529,1.538275,1.040176,1.040550,2.033352
7,2020-02-12,1.0,38.065884,2.123833,1.038480,1.036039,0.819433
8,2020-02-13,1.0,6.000317,2.251269,1.040395,1.035151,0.604910
9,2020-02-14,1.0,0.000000,2.251269,1.037357,1.029309,0.604910


In [42]:
#hide_input
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 

alt.LayerChart(...)

**Short Portfolio**

In [43]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [44]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [45]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a
0,2020-02-03,-1.0,2.859980,0.971400,1.014980,1.014360,0.860672
1,2020-02-04,-1.0,4.235264,0.930259,1.026400,1.031375,0.403449
2,2020-02-05,-1.0,-0.357010,0.933580,1.029813,1.034506,0.199411
3,2020-02-06,-1.0,-4.639900,0.976897,1.024251,1.024743,0.307143
4,2020-02-07,-1.0,11.911285,0.860536,1.031755,1.030881,0.241077
5,2020-02-10,-1.0,-6.108748,0.913104,1.033497,1.030864,0.296243
6,2020-02-11,-1.0,7.472002,0.844877,1.040176,1.040550,0.233879
7,2020-02-12,-1.0,-2.351826,0.864747,1.038480,1.036039,0.378704
8,2020-02-13,-1.0,-4.692604,0.905326,1.040395,1.035151,0.664572
9,2020-02-14,-1.0,0.000000,0.905326,1.037357,1.029309,0.664572


In [46]:
#hide_input
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4  

alt.LayerChart(...)